In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from pylab import rcParams
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import hamming_loss
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.metrics import jaccard_score
from sklearn.metrics import log_loss
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import zero_one_loss
from sklearn.metrics import brier_score_loss
from sklearn.metrics import balanced_accuracy_score

In [ ]:
df = pd.read_csv("/kaggle/input/richters-predictor-modeling-earthquake-damage/train_values.csv")

In [ ]:
df.head()

In [ ]:
df.columns

> #  Description
* > geo_level_1_id, geo_level_2_id, geo_level_3_id (type: int): geographic region in which building exists, from largest (level 1) to most specific sub-region (level 3). Possible values: level 1: 0-30, level 2: 0-1427, level 3: 0-12567.
* > count_floors_pre_eq (type: int): number of floors in the building before the earthquake.
* > age (type: int): age of the building in years.
* > area_percentage (type: int): normalized area of the building footprint.
* > height_percentage (type: int): normalized height of the building footprint.
* > land_surface_condition (type: categorical): surface condition of the land where the building was built. Possible values: n, o, t.
* > foundation_type (type: categorical): type of foundation used while building. Possible values: h, i, r, u, w.
* > roof_type (type: categorical): type of roof used while building. Possible values: n, q, x.
* > ground_floor_type (type: categorical): type of the ground floor. Possible values: f, m, v, x, z.
* > other_floor_type (type: categorical): type of constructions used in higher than the ground floors (except of roof). Possible values: j, q, s, x.
* > position (type: categorical): position of the building. Possible values: j, o, s, t.
* > plan_configuration (type: categorical): building plan configuration. Possible values: a, c, d, f, m, n, o, q, s, u.
* > has_superstructure_adobe_mud (type: binary): flag variable that indicates if the superstructure was made of Adobe/Mud.
* > has_superstructure_mud_mortar_stone (type: binary): flag variable that indicates if the superstructure was made of Mud Mortar - Stone.
*  > has_superstructure_stone_flag (type: binary): flag variable that indicates if the superstructure was made of Stone.
*   > has_superstructure_cement_mortar_stone (type: binary): flag variable that indicates if the superstructure was made of Cement Mortar - Stone.
*   > has_superstructure_mud_mortar_brick (type: binary): flag variable that indicates if the superstructure was made of Mud Mortar - Brick.
*   > has_superstructure_cement_mortar_brick (type: binary): flag variable that indicates if the superstructure was made of Cement Mortar - Brick.
*   > has_superstructure_timber (type: binary): flag variable that indicates if the superstructure was made of Timber.
*   > has_superstructure_bamboo (type: binary): flag variable that indicates if the superstructure was made of Bamboo.
*   > has_superstructure_rc_non_engineered (type: binary): flag variable that indicates if the superstructure was made of non-engineered reinforced concrete.
*   > has_superstructure_rc_engineered (type: binary): flag variable that indicates if the superstructure was made of engineered reinforced concrete.
*   > has_superstructure_other (type: binary): flag variable that indicates if the superstructure was made of any other material.
*   > legal_ownership_status (type: categorical): legal ownership status of the land where building was built. Possible values: a, r, v, w.
*   > count_families (type: int): number of families that live in the building.
*   > has_secondary_use (type: binary): flag variable that indicates if the building was used for any secondary purpose.
*   > has_secondary_use_agriculture (type: binary): flag variable that indicates if the building was used for agricultural purposes.
*   > has_secondary_use_hotel (type: binary): flag variable that indicates if the building was used as a hotel.
*   > has_secondary_use_rental (type: binary): flag variable that indicates if the building was used for rental purposes.
*   > has_secondary_use_institution (type: binary): flag variable that indicates if the building was used as a location of any institution.
*   > has_secondary_use_school (type: binary): flag variable that indicates if the building was used as a school.
*   > has_secondary_use_industry (type: binary): flag variable that indicates if the building was used for industrial purposes.
*   > has_secondary_use_health_post (type: binary): flag variable that indicates if the building was used as a health post.
*   > has_secondary_use_gov_office (type: binary): flag variable that indicates if the building was used fas a government office.
*   > has_secondary_use_use_police (type: binary): flag variable that indicates if the building was used as a police station.
*   > has_secondary_use_other (type: binary): flag variable that indicates if the building was secondarily used for other purposes.

In [ ]:
df.shape

In [ ]:
df2 = pd.read_csv("/kaggle/input/richters-predictor-modeling-earthquake-damage/train_labels.csv")

In [ ]:
df2.head()

In [ ]:
df2.shape

In [ ]:
df['damage_grade'] = df2['damage_grade']

In [ ]:
plt.figure(figsize=(10,10))
cor=df.corr()["damage_grade"]
cor=pd.DataFrame(cor)
sns.heatmap(cor,annot=True,cmap="viridis")

In [ ]:
# exploring the statstical fearures of train dataset.
df.describe().T.style.background_gradient(cmap='Set2',low =0.4,high=0.1,axis=0)

In [ ]:
df.describe(include="object").T.style.background_gradient(cmap='Set2',axis=0)


In [ ]:
# checking the types of varibles in the dataset(int,float,object)
dtypes=pd.DataFrame(df.dtypes,columns=["Data Type"])
dtypes["Unique Values"]=df.nunique()
dtypes["Null Values"]=df.isnull().sum()
dtypes["% null Values"]=df.isnull().sum()/len(df)
dtypes.style.background_gradient(cmap='Set2',axis=0)

> We're trying to predict the ordinal variable damage_grade, which represents a level of damage to the building that was hit by the earthquake. There are 3 grades of the damage
> 
1. > 1: represents low damage
2. > 2: represents a medium amount of damage
3. > 3:
represents almost complete destruction

In [ ]:
y = df2['damage_grade']

In [ ]:
y.unique()

In [ ]:
y.value_counts()

In [ ]:
df.isna().any()

In [ ]:
df.info()

In [ ]:
df["has_secondary_use_hotel"].unique()
df["has_secondary_use_other"].unique()


In [ ]:
col = df.columns
col = df[['has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'legal_ownership_status', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
       'has_secondary_use_health_post', 'has_secondary_use_gov_office',
       'has_secondary_use_use_police', 'has_secondary_use_other']]

In [ ]:
for i in col:
    print (df[i].unique())

In [ ]:
df[['has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'legal_ownership_status', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
       'has_secondary_use_health_post', 'has_secondary_use_gov_office',
       'has_secondary_use_use_police', 'has_secondary_use_other']] = df[['has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'legal_ownership_status', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
       'has_secondary_use_health_post', 'has_secondary_use_gov_office',
       'has_secondary_use_use_police', 'has_secondary_use_other']].astype('object')

In [ ]:
df.info()

In [ ]:
l0 = df['building_id'].unique()
l1 = df['geo_level_1_id'].unique()
l2 = df['geo_level_2_id'].unique()
l3 = df['geo_level_3_id'].unique()

In [ ]:
len(l1)
len(l2)
len(l3)
len(l0)#all unique

In [ ]:
df1 = df[['count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'legal_ownership_status', 'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
       'has_secondary_use_health_post', 'has_secondary_use_gov_office',
       'has_secondary_use_use_police', 'has_secondary_use_other']]

In [ ]:
df1.info()

In [ ]:
X = pd.DataFrame(df1)

In [ ]:
X = pd.get_dummies(X,drop_first=True)

> # GENERATE MODEL REPORT

In [ ]:
def generate_model_report(y_actual, y_predicted):
    print("Accuracy = " , accuracy_score(y_actual, y_predicted))
    print("Precision = " ,precision_score(y_actual, y_predicted,average="micro"))
    print("Recall = " ,recall_score(y_actual, y_predicted,average="micro"))
    print("F1 Score = " ,f1_score(y_actual, y_predicted,average="micro"))
    print("Cohen Kappa score= ",cohen_kappa_score(y_actual,y_predicted))
    print("Hamming Loss = ",hamming_loss(y_actual,y_predicted))
    print("Jaccard Score = ",jaccard_score(y_actual, y_predicted,average="micro"))
#     print("Log Loss = ",log_loss(y_actual, y_predicted))
    print("Matthews correlation coefficient = ", matthews_corrcoef(y_actual, y_predicted))
    print("Zero one loss = ",zero_one_loss(y_actual, y_predicted))
#     print("brier_score_loss  = ",brier_score_loss(y_actual, y_predicted))
    print("Balanced accuracy = ",balanced_accuracy_score(y_actual, y_predicted))

> # Generate auc roc curve

In [ ]:
def generate_auc_roc_curve(clf, X_test):
    y_pred_proba = clf.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test,  y_pred_proba)
    auc = roc_auc_score(y_test, y_pred_proba)
    plt.plot(fpr,tpr,label="AUC ROC Curve with Area Under the curve ="+str(auc))
    plt.legend(loc=4)
    plt.show()
    pass

> # Normalization

In [ ]:
from sklearn import preprocessing
# from sklearn.preprocessing import Normalizer
X = preprocessing.normalize(X, norm='l2')

In [ ]:
X.shape

In [ ]:
y = y.astype('int')


> # Train test split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

> # Generate auc_roc_curve

In [ ]:
def generate_auc_roc_curve(clf, X_test):
    y_pred_proba = clf.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test,  y_pred_proba)
    auc = roc_auc_score(y_test, y_pred_proba,multi_class = "ovr",average = "macro")
    plt.plot(fpr,tpr,label="AUC ROC Curve with Area Under the curve ="+str(auc))
    plt.legend(loc=4)
    plt.show()
    pass



> # LOGISTIC REGRESSION

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state = 0, multi_class = "multinomial", solver="newton-cg")
lr.get_params()

In [ ]:
lr.fit(X_train,y_train)

In [ ]:
y_lr = lr.predict(X_test)
y_lr = pd.DataFrame(y_lr)
y_lr.value_counts()

In [ ]:
generate_model_report(y_test, y_lr)

In [ ]:
y_prob = lr.predict_proba(X_test)
y_prob

In [ ]:
macro_roc_auc_ovo = roc_auc_score(y_test, y_prob, multi_class="ovo",
                                  average="macro")
weighted_roc_auc_ovo = roc_auc_score(y_test, y_prob, multi_class="ovo",
                                     average="weighted")
macro_roc_auc_ovr = roc_auc_score(y_test, y_prob, multi_class="ovr",
                                  average="macro")
weighted_roc_auc_ovr = roc_auc_score(y_test, y_prob, multi_class="ovr",
                                     average="weighted")
print("One-vs-One ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
      "(weighted by prevalence)"
      .format(macro_roc_auc_ovo, weighted_roc_auc_ovo))
print("One-vs-Rest ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
      "(weighted by prevalence)"
      .format(macro_roc_auc_ovr, weighted_roc_auc_ovr))